In [1]:
from rsome import ro
from rsome import norm
import numpy as np
import pandas as pd
from rsome import grb_solver as grb
import gurobipy

In [5]:
df1

,ID,Large B/P,Large ROE,Large S/P,Large Return Rate in the last quarter,Large Market Value,Small systematic Risk,Annual Return,Excess Return,Systematic Risk,Total Risk,Abs. Win Rate,Rel. Win Rate
0,1,1.000,0.000,0.000,0.000,0.000,0.000,25.54%,2.08%,0.975575,8.14%,80.00%,60.00%
1,2,0.000,1.000,0.000,0.000,0.000,0.000,24.96%,0.40%,1.350801,7.75%,85.00%,75.00%
2,3,0.000,0.000,1.000,0.000,0.000,0.000,23.12%,1.35%,1.040006,8.78%,80.00%,40.00%
3,4,0.000,0.000,0.000,1.000,0.000,0.000,20.38%,-0.26%,1.291899,8.86%,70.00%,60.00%
4,5,0.000,0.000,0.000,0.000,1.000,0.000,16.38%,-0.08%,0.974706,4.61%,80.00%,45.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,59,0.200,0.200,0.200,0.000,0.200,0.200,27.65%,2.05%,1.069434,6.60%,80.00%,65.00%
59,60,0.200,0.200,0.000,0.200,0.200,0.200,23.07%,1.16%,1.043684,6.05%,80.00%,70.00%
60,61,0.200,0.000,0.200,0.200,0.200,0.200,23.47%,2.29%,0.794887,6.29%,80.00%,55.00%
61,62,0.000,0.200,0.200,0.200,0.200,0.200,24.29%,1.13%,1.125114,6.77%,80.00%,80.00%


In [2]:
df = pd.read_csv('period1.csv')

In [21]:
def process_dataset(path):
    df = pd.read_csv(path, index_col= 0)
    df['Annual Return'] = df['Annual Return'].apply(lambda x : float(x[:-1]))
    df['Total Risk'] = df['Total Risk'].apply(lambda x : float(x[:-1]))
    df['Abs. Win Rate'] = df['Abs. Win Rate'].apply(lambda x : 100 - float(x[:-1]))
    if(min(df['Annual Return']) < 0):
        df['Annual Return'] -= min(df['Annual Return'])
    X = np.array((df[['Total Risk','Abs. Win Rate']]))
    Y = np.array((df[['Annual Return']]))
    return X,Y

    


In [22]:
X1,Y1 = process_dataset('period1.csv')
X2,Y2 =process_dataset('period2.csv')
X3,Y3 = process_dataset('period3.csv')
X4,Y4 = process_dataset('period4.csv')

In [23]:
X1

array([[ 8.14, 20.  ],
       [ 7.75, 15.  ],
       [ 8.78, 20.  ],
       [ 8.86, 30.  ],
       [ 4.61, 20.  ],
       [ 8.07, 25.  ],
       [ 6.49, 10.  ],
       [ 8.34, 20.  ],
       [ 7.72, 25.  ],
       [ 5.98, 15.  ],
       [ 8.49, 20.  ],
       [ 7.18, 20.  ],
       [ 5.73, 15.  ],
       [ 6.29, 30.  ],
       [ 5.72, 15.  ],
       [ 6.2 , 30.  ],
       [ 7.27, 10.  ],
       [ 8.7 , 15.  ],
       [ 8.09, 20.  ],
       [ 8.95, 25.  ],
       [ 5.48, 30.  ],
       [ 7.13, 20.  ],
       [ 6.63, 20.  ],
       [ 7.19, 15.  ],
       [ 7.77, 20.  ],
       [ 5.96, 25.  ],
       [ 6.27, 20.  ],
       [ 6.28, 30.  ],
       [ 4.77, 20.  ],
       [ 7.06, 30.  ],
       [ 5.14, 20.  ],
       [ 6.88, 20.  ],
       [ 7.87, 20.  ],
       [ 8.09, 20.  ],
       [ 6.89, 20.  ],
       [ 8.56, 25.  ],
       [ 7.93, 25.  ],
       [ 6.48, 15.  ],
       [ 7.03, 30.  ],
       [ 6.89, 20.  ],
       [ 7.36, 35.  ],
       [ 7.31, 25.  ],
       [ 6.13, 15.  ],
       [ 5.

In [52]:
X1_ag = []
X2_ag = []
Y1_ag = []

In [53]:
X1_ag.append(np.array(X1[:,0]))
X1_ag.append(np.array(X2[:,0]))
X1_ag.append(np.array(X3[:,0]))  
X1_ag.append(np.array(X4[:,0]))
X2_ag.append((np.array(X1[:,1])))
X2_ag.append(np.array(X2[:,1]))
X2_ag.append(np.array(X3[:,1]))
X2_ag.append(np.array(X4[:,1]))
Y1_ag.append((np.array(Y1[:,0])))
Y1_ag.append(np.array(Y2[:,0]))
Y1_ag.append(np.array(Y3[:,0]))
Y1_ag.append(np.array(Y4[:,0]))

In [68]:
Avg = np.array([np.mean(np.array(X1_ag),  axis=0),np.mean(np.array(X2_ag),  axis=0) ,np.mean(np.array(Y1_ag),  axis=0)])

In [72]:
df_out = pd.DataFrame(columns= ['Annual Return','Total Risk', 'Abs. Loss Rate' ], data= Avg.T)


In [74]:
df_out.to_csv("periodall.csv")

In [65]:
Avg

array([14.0075, 10.75  , 13.88  , 14.2025,  8.4175, 10.705 , 10.07  ,
       14.15  , 10.7725, 10.83  , 11.42  , 12.005 , 10.1825,  8.7425,
        9.9975, 10.63  , 10.475 ,  9.9775, 11.13  , 12.0225,  8.74  ,
       11.3025, 10.0125, 11.9275, 10.8025,  8.96  , 11.6475,  9.22  ,
        9.1925,  9.93  ,  9.6975,  9.175 , 11.575 , 10.33  ,  9.945 ,
       10.21  , 11.285 ,  9.6675,  8.7325,  9.7675, 11.0175, 11.0475,
       10.0175,  9.1225, 10.47  ,  9.6925, 10.6475,  9.235 , 11.465 ,
       10.47  ,  8.3825, 10.5275,  9.1275,  8.8375,  9.5975,  9.7425,
        9.8525, 10.3475,  9.705 ,  8.5625, 10.1375,  9.355 ,  9.3175])

In [55]:
X_av = [Y1_ag,X1_ag,X2_ag]

In [58]:
X_av

[[array([25.54, 24.96, 23.12, 20.38, 16.38, 17.35, 29.49, 25.59, 24.82,
         22.76, 26.13, 22.83, 24.39, 18.45, 21.53, 15.96, 23.85, 27.56,
         23.51, 18.67, 18.25, 29.01, 24.91, 23.77, 25.81, 22.07, 25.53,
         21.13, 18.81, 20.61, 18.94, 28.35, 24.73, 27.56, 22.04, 25.45,
         23.98, 25.08, 22.41, 22.82, 15.95, 26.67, 25.  , 20.38, 22.4 ,
         20.79, 28.35, 26.07, 23.69, 27.88, 26.05, 26.1 , 24.83, 19.38,
         22.37, 20.58, 23.24, 27.36, 27.65, 23.07, 23.47, 24.29, 24.16]),
  array([ 7.7 , 16.75,  9.76, 20.34, 17.85,  8.36, 14.39,  8.65, 14.22,
         11.22, 22.03, 11.82, 12.1 , 18.26, 13.72, 18.55,  7.01, 15.3 ,
          8.86, 15.47, 13.27, 11.77, 16.66, 10.9 , 17.45, 13.65, 11.75,
         14.12, 12.3 , 20.28, 12.18, 12.16,  7.1 , 14.94, 11.51, 19.47,
         14.04,  9.95, 17.67, 10.75, 19.03, 13.7 , 12.8 , 17.23, 10.14,
         17.78, 11.99, 16.21, 10.36, 15.67, 13.89, 10.03, 14.98, 13.06,
         21.92, 12.48, 13.87, 14.11, 11.64, 16.06, 11.47, 17.0

In [48]:
X_av = np.array(X_av)
Y_av = np.array(Y_av)

In [57]:

file = open("arr", "wb")
np.save(file, X_av)
file.close

<function BufferedWriter.close>

In [43]:
X_av.T

array([[14.0075, 33.75  ],
       [10.75  , 33.75  ],
       [13.88  , 36.25  ],
       [14.2025, 38.75  ],
       [ 8.4175, 27.5   ],
       [10.705 , 43.75  ],
       [10.07  , 26.25  ],
       [14.15  , 35.    ],
       [10.7725, 32.5   ],
       [10.83  , 30.    ],
       [11.42  , 32.5   ],
       [12.005 , 31.25  ],
       [10.1825, 30.    ],
       [ 8.7425, 35.    ],
       [ 9.9975, 30.    ],
       [10.63  , 33.75  ],
       [10.475 , 31.25  ],
       [ 9.9775, 32.5   ],
       [11.13  , 36.25  ],
       [12.0225, 38.75  ],
       [ 8.74  , 37.5   ],
       [11.3025, 30.    ],
       [10.0125, 27.5   ],
       [11.9275, 30.    ],
       [10.8025, 30.    ],
       [ 8.96  , 28.75  ],
       [11.6475, 32.5   ],
       [ 9.22  , 32.5   ],
       [ 9.1925, 31.25  ],
       [ 9.93  , 32.5   ],
       [ 9.6975, 32.5   ],
       [ 9.175 , 26.25  ],
       [11.575 , 36.25  ],
       [10.33  , 32.5   ],
       [ 9.945 , 33.75  ],
       [10.21  , 35.    ],
       [11.285 , 35.    ],
 

In [7]:
def getABC(X,Y,i):
    X_new = np.array(X).T #N X D
    D = len(X_new[0])
    Y_new = np.array(Y).T# M X D
    y_i = Y_new[:,[i]] # in paper i from 1 to n and we won't follow it
    x_i = X_new[:,[i]] # in paper i from 1 to n and we won't follow it
    up = np.concatenate((-Y_new,y_i, np.zeros((len(Y_new),1))),axis=1)
    low = np.concatenate((X_new,np.zeros((len(X_new),1)),-x_i),axis=1)
    A = np.concatenate((up,low), axis = 0)
    B = np.concatenate((np.ones((1,D)),np.zeros((1,1)), np.zeros((1,1))), axis = 1)
    B = np.concatenate((B, np.zeros((1, D + 2))), axis = 0)
    B[1,-2] = 1
    C = np.zeros((D+2,1))
    C[-1,0] = 1
    return A, B, C

def DEA_eff(X,Y):
    eff_arr = []
    D = len(X)
    for i in range(D):
        A,B,C = getABC(X,Y,i)
        '''
        if (not sigma):
            sigma = [0.5]*len(A[0])
        '''
        model = ro.Model()
        neta = model.dvar((len(X)+2))
        #z = [model.rvar((len(X))) for i in range(len(X[0]) + len(Y[0])) ]
        #z_set0 = (norm(z[i],2) <= 1 for i in range(len(X[0]) + len(Y[0])) )
        model.min((C.T)[0]@neta)
        model.st(B[0]@ neta == 1,B[1] @ neta == 1 )
        model.st(neta >= (np.zeros(len(X)+2)))
        for x in range(len(A)):
            A_x = A[x]
            I = np.eye(len(A[0]))
            model.st(((A_x)@neta <= 0))
            #model.st(((A_x)@neta <= 0))
        try:
            model.solve(grb,  display = False)
        except gurobipy.GurobiError:
            return -1
        try :
            eff = (neta.get())[-1]
        except RuntimeError:
            return -1
        eff_arr.append(eff)
    return eff_arr

In [8]:
X, Y = process_dataset('period1.csv')

In [9]:
eff = DEA_eff(X,Y)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-16


In [10]:
eff

[0.7024753281659358,
 0.7321428571428572,
 0.625673249551167,
 0.5681629162351316,
 1.0,
 0.6324863883847549,
 1.0,
 0.6864740251997298,
 0.7275534594795751,
 0.900179872454078,
 0.6838962390774371,
 0.7432247341448777,
 0.9696566486361401,
 0.7545781075192839,
 0.9305740987983978,
 0.7435483870967742,
 1.0,
 0.7008353381448811,
 0.6719329479665098,
 0.5857142857142857,
 0.8637094109519088,
 0.8965393522293618,
 0.8490787730306742,
 0.7779017857142857,
 0.7408221165127243,
 0.8773962697912417,
 0.911760818419731,
 0.8116004504345989,
 1.0,
 0.7115568215610473,
 0.9419520706383091,
 0.9095964034613306,
 0.7120753683826264,
 0.7536795354586484,
 0.7608844818424298,
 0.6665268239789333,
 0.6933631987069176,
 0.8724294045426642,
 0.7506660484270254,
 0.7743025763978542,
 0.626358695652174,
 0.8093245951706806,
 0.9204032338016999,
 0.8590684759559152,
 0.9751891831170023,
 0.8094215436999616,
 0.8245089928608635,
 0.8446165841751981,
 0.7218933613961238,
 0.7741688091620896,
 1.0,
 0.86056

In [6]:
Y

[[25.54],
 [24.96],
 [23.12],
 [20.38],
 [16.38],
 [17.35],
 [29.49],
 [25.59],
 [24.82],
 [22.76],
 [26.13],
 [22.83],
 [24.39],
 [18.45],
 [21.53],
 [15.96],
 [23.85],
 [27.56],
 [23.51],
 [18.67],
 [18.25],
 [29.01],
 [24.91],
 [23.77],
 [25.81],
 [22.07],
 [25.53],
 [21.13],
 [18.81],
 [20.61],
 [18.94],
 [28.35],
 [24.73],
 [27.56],
 [22.04],
 [25.45],
 [23.98],
 [25.08],
 [22.41],
 [22.82],
 [15.95],
 [26.67],
 [25.0],
 [20.38],
 [22.4],
 [20.79],
 [28.35],
 [26.07],
 [23.69],
 [27.88],
 [26.05],
 [26.1],
 [24.83],
 [19.38],
 [22.37],
 [20.58],
 [23.24],
 [27.36],
 [27.65],
 [23.07],
 [23.47],
 [24.29],
 [24.16]]

In [7]:
eng = matlab.engine.start_matlab()

In [8]:
X = matlab.double(X)

Y = matlab.double(Y)

In [9]:
Y

matlab.double([[25.54],[24.96],[23.12],[20.38],[16.38],[17.35],[29.49],[25.59],[24.82],[22.76],[26.13],[22.83],[24.39],[18.45],[21.53],[15.96],[23.85],[27.56],[23.51],[18.67],[18.25],[29.01],[24.91],[23.77],[25.81],[22.07],[25.53],[21.13],[18.81],[20.61],[18.94],[28.35],[24.73],[27.56],[22.04],[25.45],[23.98],[25.08],[22.41],[22.82],[15.95],[26.67],[25.0],[20.38],[22.4],[20.79],[28.35],[26.07],[23.69],[27.88],[26.05],[26.1],[24.83],[19.38],[22.37],[20.58],[23.24],[27.36],[27.65],[23.07],[23.47],[24.29],[24.16]])

In [10]:
io_vrs_1 = eng.dea(X,Y, 'orient', 'io', 'rts', 'vrs') #dea(X, Y, 'orient', 'io', 'rts', 'vrs');

In [11]:
efficiency = np.asarray(io_vrs_1['eff'])

In [12]:
efficiency

array([[0.70247533],
       [0.73214286],
       [0.62567325],
       [0.56816292],
       [1.        ],
       [0.63248639],
       [1.        ],
       [0.68647403],
       [0.72755346],
       [0.90017987],
       [0.68389624],
       [0.74322473],
       [0.96965665],
       [0.75457811],
       [0.9305741 ],
       [0.74354839],
       [1.        ],
       [0.70083534],
       [0.67193295],
       [0.58571429],
       [0.86370941],
       [0.89653935],
       [0.84907877],
       [0.77790179],
       [0.74082212],
       [0.87739627],
       [0.91176082],
       [0.81160045],
       [1.        ],
       [0.71155682],
       [0.94195207],
       [0.9095964 ],
       [0.71207537],
       [0.75367954],
       [0.76088448],
       [0.66652682],
       [0.6933632 ],
       [0.8724294 ],
       [0.75066605],
       [0.77430258],
       [0.6263587 ],
       [0.8093246 ],
       [0.92040323],
       [0.85906848],
       [0.97518918],
       [0.80942154],
       [0.82450899],
       [0.844

In [13]:
eng.quit()

In [14]:
efficiency

array([[0.70247533],
       [0.73214286],
       [0.62567325],
       [0.56816292],
       [1.        ],
       [0.63248639],
       [1.        ],
       [0.68647403],
       [0.72755346],
       [0.90017987],
       [0.68389624],
       [0.74322473],
       [0.96965665],
       [0.75457811],
       [0.9305741 ],
       [0.74354839],
       [1.        ],
       [0.70083534],
       [0.67193295],
       [0.58571429],
       [0.86370941],
       [0.89653935],
       [0.84907877],
       [0.77790179],
       [0.74082212],
       [0.87739627],
       [0.91176082],
       [0.81160045],
       [1.        ],
       [0.71155682],
       [0.94195207],
       [0.9095964 ],
       [0.71207537],
       [0.75367954],
       [0.76088448],
       [0.66652682],
       [0.6933632 ],
       [0.8724294 ],
       [0.75066605],
       [0.77430258],
       [0.6263587 ],
       [0.8093246 ],
       [0.92040323],
       [0.85906848],
       [0.97518918],
       [0.80942154],
       [0.82450899],
       [0.844